# Get Data

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime
import requests
import json


CLIENT_ID='23QQNX'
CLIENT_SECRET='2cf8156ff7b81eadd26d80a30082067c'

# API Authorization

In [13]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[23/Apr/2023:13:04:07] ENGINE Listening for SIGTERM.
[23/Apr/2023:13:04:07] ENGINE Bus STARTING
[23/Apr/2023:13:04:07] ENGINE Set handler for console events.
[23/Apr/2023:13:04:07] ENGINE Started monitor thread 'Autoreloader'.
[23/Apr/2023:13:04:07] ENGINE Serving on http://127.0.0.1:8080
[23/Apr/2023:13:04:07] ENGINE Bus STARTED


127.0.0.1 - - [23/Apr/2023:13:04:09] "GET /?code=9a37e133f58850bec062baa73e40ff1b81683d53&state=kJjcU6EV36HqLn64SivdFOxJPk7hhF HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"


[23/Apr/2023:13:04:10] ENGINE Bus STOPPING
[23/Apr/2023:13:04:10] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[23/Apr/2023:13:04:10] ENGINE Stopped thread 'Autoreloader'.
[23/Apr/2023:13:04:10] ENGINE Removed handler for console events.
[23/Apr/2023:13:04:10] ENGINE Bus STOPPED
[23/Apr/2023:13:04:10] ENGINE Bus EXITING
[23/Apr/2023:13:04:10] ENGINE Waiting for child threads to terminate...
[23/Apr/2023:13:04:10] ENGINE Bus EXITED
[23/Apr/2023:13:04:10] ENGINE Waiting for thread Thread-32.


# Get Data

### Automated process with a function to access each of the variables

In [14]:
def df_fitbit(activity, base_date, end_date, token):
    url = 'https://api.fitbit.com/1/user/-/' + activity + '/date/' + base_date + '/' + end_date + '.json'
    response = requests.get(url=url, headers={'Authorization':'Bearer ' + token}).json()

    return response


In [27]:
token = ACCESS_TOKEN
base_date = '2023-03-29'
end_date = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

#data to gather
activity = ['activities/minutesSedentary', 'activities/log/minutesLightlyActive', 'activities/log/minutesFairlyActive', 
      'activities/log/minutesVeryActive','activities/steps']

sleep = ['sleep/startTime', 'sleep/timeInBed', 'sleep/minutesAsleep', 'sleep/minutesAwake', 'sleep/efficiency']

### Get User Engagement Data

In [24]:
minutesSedentary = df_fitbit(activity[0], base_date, end_date, token)
#print(minutesSedentary)
minutesLightlyActive = df_fitbit(activity[1], base_date, end_date, token)
#print(minutesLightlyActive)
minutesFairlyActive = df_fitbit(activity[2], base_date, end_date, token)
#print(minutesFairlyActive)
minutesVeryActive = df_fitbit(activity[3], base_date, end_date, token)
#print(minutesVeryActive)

In [32]:
#data to gather (in that case, steps)
getSteps = df_fitbit(activity[4], base_date, end_date, token)
#print(getSteps)

### Get Sleep Data

In [17]:
StartTime = df_fitbit(sleep[0], base_date, end_date, token)
#print(StartTime)
timeInBed = df_fitbit(sleep[1], base_date, end_date, token)
#print(timeInBed)
minutesAsleep = df_fitbit(sleep[2], base_date, end_date, token)
#print(minutesAsleep)
minutesAwake = df_fitbit(sleep[3], base_date, end_date, token)
#print(minutesAwake)
efficiency = df_fitbit(sleep[4], base_date, end_date, token)
#print(efficiency)

### Transform Data into the appropriate form

In [18]:
import hashlib
import random

def create_data(dicts,types):
    data_list = []
    # loop through each nested dictionary
    for i in range(0,len(dicts)):        
        for item in dicts[i]:            
            # create a random hash ID
            hash_object = hashlib.sha256(str(random.getrandbits(256)).encode())
            id = hash_object.hexdigest()           
            # create a new dictionary with the id, type, and data fields
            new_dict = {
                'id': id,
                'type': types[i],
                'data': item
            }           
            # append the new dictionary to the output list
            data_list.append(new_dict)
    return data_list     

In [31]:
#gather all activity dictionaries in a list
act_dicts = [minutesSedentary['activities-minutesSedentary'], minutesLightlyActive['activities-log-minutesLightlyActive'], 
              minutesFairlyActive['activities-log-minutesFairlyActive'], minutesVeryActive['activities-log-minutesVeryActive'],
              getSteps['activities-steps']]
act_types = ['minutesSedentary', 'minutesLightlyActive', 'minutesFairlyActive', 'minutesVeryActive','steps']

activity_data = create_data(act_dicts,act_types)
activity_data

[{'id': '96e62b92c08857f371495f9e0dfb715a0a11b7378255065cbd24bb79cc6558b1',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-03-29', 'value': '727'}},
 {'id': '41d8ab0d1785a05c9990fba173fe38b44fe1755d3fb5710f5375c6b790c6edc6',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-03-30', 'value': '776'}},
 {'id': '86e2b29be4df15bfc79c549adf52d5b22164dc2baf41e70a559b28f79c925b83',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-03-31', 'value': '730'}},
 {'id': '3deebebbd4aa2bd8dd9eddb8a58d0def3d0c0a09219ed75c3e9c506fdbdba60e',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-04-01', 'value': '753'}},
 {'id': '82b156c04fc34784c91f461c86146fb0fa11882f0e89f8b88ca22504b33506ae',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-04-02', 'value': '745'}},
 {'id': '9353d46fd40ea61e4c2d1fe3ab9ca83a2b20492caf6583fb7c9fc4731a1ee9b8',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-04-03', 'value': '823'}},
 {'id': 'a3430823ef3305f3ce567b236

In [20]:
#gather all sleep dictionaries in a list
sleep_dicts = [StartTime['sleep-startTime'], timeInBed['sleep-timeInBed'], minutesAsleep['sleep-minutesAsleep'], 
             minutesAwake['sleep-minutesAwake'], efficiency['sleep-efficiency']]
sleep_types = ['startTime', 'timeInBed', 'minutesAsleep', 'minutesAwake','efficiency']

sleep_data = create_data(sleep_dicts,sleep_types)
sleep_data

[{'id': '4c7592a6d74f07ddaf0e6299bf1368ea283e7837ccb85f5845112c06cd55dcd8',
  'type': 'startTime',
  'data': {'dateTime': '2023-03-29', 'value': '00:34'}},
 {'id': 'd1a4f7ffd8862792d7f844c38fdd47593e0f722fce6f09b9631e942de8c4313a',
  'type': 'startTime',
  'data': {'dateTime': '2023-03-30', 'value': '00:52'}},
 {'id': 'd43d5f61cee5067322e8596afba47498f8b91efa28ff19064a710a91e3a06d09',
  'type': 'startTime',
  'data': {'dateTime': '2023-03-31', 'value': '00:23'}},
 {'id': '590705563770e7f63004437acd2542df0610254050bce89aea2859386e9b2269',
  'type': 'startTime',
  'data': {'dateTime': '2023-04-01', 'value': '03:01'}},
 {'id': '4a28e2b6adc4eaf359f8adbd54ec9a3e76bfba480cadadb1090bd2751bd15261',
  'type': 'startTime',
  'data': {'dateTime': '2023-04-02', 'value': '05:08'}},
 {'id': 'd50fac3879756363a2bba4151bafde473dac103d31ae086163a076e5b78a2ffc',
  'type': 'startTime',
  'data': {'dateTime': '2023-04-03', 'value': '23:28'}},
 {'id': 'c64a94d0912c5ca7f6aae680a65f15eff48e457ffcaf1719ce887d5

# Inserting data into MongoDB 

In [21]:
# importing pymongo
from pymongo import MongoClient

# establing connection
try:
    connect = MongoClient()
    print("Connected successfully!!!")
except:
    print("Could not connect to MongoDB")

# connecting or switching to the database
db = connect.fitbitDB

# creating or switching to demoCollection
collection = db.demoCollection

Connected successfully!!!


In [22]:
# Inserting activity documents one by one
for document in activity_data:
    collection.insert_one(document)

In [23]:
# Inserting sleep documents one by one
for document in sleep_data:
    collection.insert_one(document)